In [58]:
import pandas as pd
import numpy as py

In [59]:
df = pd.read_csv("hotel_bookings_cleaned.csv")

In [60]:
#there's no Identifier Id so we create this synthetic guest ID 
df['customer_id'] = (
    df['country'].astype(str) + '_' +
    df['market_segment'].astype(str) + '_' +
    df['distribution_channel'].astype(str) + '_' +
    df['reserved_room_type'].astype(str)
).factorize()[0] + 1 #factorize in pandas means assign unique id integer to each distinct combination

In [63]:
#loyality check 

# checks if the guest id appears more than once in the dataset
df['is_repeated_guest'] = df['customer_id'].duplicated(keep=False).astype(int)

# checks if the guest cancels before
guest_cancel = df.groupby('customer_id')['is_canceled'].max().to_dict() 
df['previous_cancellations'] = df['customer_id'].map(guest_cancel)

#combine both checks with OR if either one of them is true - if the customer repeated or have previous cancellation their loyality check is 1, otherwise 0
df['loyal_check'] = ((df['is_repeated_guest'] == 1) | (df['previous_cancellations'] == 1)).astype(int)

print(df[['customer_id', 'is_repeated_guest','previous_cancellations','loyal_check']].head())

   customer_id  is_repeated_guest  previous_cancellations  loyal_check
0            1                  1                       1            1
1            1                  1                       1            1
2            2                  1                       1            1
3            3                  1                       0            1
4            4                  1                       1            1


In [62]:
#customer segments
def customer_segment(seg):
    all_kids = seg['children'] + seg['babies']
    
    if seg['adults'] > 1 and all_kids > 0:
        return "Family"
    elif seg['adults'] == 2 and all_kids == 0:
        return "Couple"
    elif seg['adults'] == 1 and all_kids == 0:
        return "Solo"
    
    else:
        return "Others"
    
#pandas function to apply thats run the function for each row and apply it to the new column which is customer segmentation.    
df['customer_segmentation'] = df.apply(customer_segment, axis=1)

print(df[['adults','children','babies','customer_segmentation']].head())

   adults  children  babies customer_segmentation
0       2         0       0                Couple
1       2         0       0                Couple
2       1         0       0                  Solo
3       1         0       0                  Solo
4       2         0       0                Couple
